In [27]:
from glob import glob
from os.path import join, split, splitext, dirname
import pandas as pd
from scipy.stats import norm
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
def GetDrawFilenames(folder):
    draw_folder = join(folder, 'draw_dicts')
    draw_filenames = glob(join(draw_folder, "*.npz"))
    if len(draw_filenames) == 0:
        raise ValueError(f'No npz files found in {draw_folder}')
    model_names = [ splitext(split(filename)[-1])[0]
                    for filename in draw_filenames ]
    return draw_filenames, model_names


In [17]:
base_path = '/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/'
input_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')
output_path = join(base_path, 'comparison/analysis/coverage_warm_starts_rerun')

M_vals = [ 8, 16, 32, 64 ]

In [63]:
M = 8
print(join(input_path, '*', '*.pkl'))
coverage_filenames = glob(join(input_path, f'M_{M}', '*', '*.pkl'))
assert(len(coverage_filenames) > 0)

# The final directory is the modle name
model_names = [ split(os.path.dirname(filename))[-1]
                for filename in coverage_filenames ]

i = 0
model_name = model_names[i]
filename = coverage_filenames[i]
raw_model_df = pd.read_pickle(filename)
num_runs = model_df.shape[0]
#model_df.iloc[0]

/home/rgiordan/Documents/git_repos/DADVI/dadvi-experiments/comparison/analysis/coverage_warm_starts_rerun/*/*.pkl


In [59]:
raw_model_df.keys()

Index(['means', 'seed', 'freq_sds', 'newton_step_norm', 'scipy_opt_result',
       'reference_means', 'reference_freq_sds', 'M',
       'reference_newton_step_norm', 'reference_scipy_opt_result'],
      dtype='object')

In [61]:
def RepList(x, n):
    return [x for _ in range(n)]


In [72]:
means = raw_model_df['means'].to_numpy()
seeds = raw_model_df['seed'].to_numpy()
freq_sds = raw_model_df['freq_sds'].to_numpy()
assert(len(means) == len(freq_sds))
num_runs = len(means) 
# This is not the most efficient or elegant but it will let me make sure
# everything lines up correctly
param_dim = len(means[0]) 

model_dict = {
    'seed': [],
    'param': [],
    'mean': [],
    'freq_sd': []
}
for i in range(num_runs):
    assert(len(means[i]) == len(freq_sds[i]))
    model_dict['seed'].append(RepList(seeds[i], param_dim))
    model_dict['param'].append(np.arange(param_dim))
    model_dict['mean'].append(means[i])
    model_dict['freq_sd'].append(freq_sds[i])

model_df = pd.DataFrame()
for k,v in model_dict.items():
    model_df[k] = np.hstack(v)
model_df['model'] = model_name
model_df['num_draws'] = M

In [73]:
model_df

,seed,param,mean,freq_sd,model,num_draws
0,1000,0,-0.823768,0.289006,radon_no_pool_chr,8
1,1000,1,-1.419175,0.138483,radon_no_pool_chr,8
2,1000,2,-0.469797,0.249086,radon_no_pool_chr,8
3,1000,3,0.213661,0.224943,radon_no_pool_chr,8
4,1000,4,0.212904,0.322659,radon_no_pool_chr,8
...,...,...,...,...,...,...
8895,1099,84,-0.459556,0.369352,radon_no_pool_chr,8
8896,1099,85,-1.106162,0.026000,radon_no_pool_chr,8
8897,1099,86,1.361113,0.006766,radon_no_pool_chr,8
8898,1099,87,-0.685845,0.026196,radon_no_pool_chr,8


In [2]:
# Which run to use as a reference; set to None to use the one initially intended to be the reference.
REFERENCE_ROW_INDEX = 1

# Set this to the appropriate directory
# The M_x means how many draws were used to run the experiments
base_path = './coverage_warm_starts_rerun/M_64/'

# Which model name to plot. Use "ARM" to analyse all ARM models.
model_name = 'occ_det'

coverage_runs = glob(join(base_path, '*', '*.pkl'))
model_names = [x.split('/')[-2] for x in coverage_runs]
assert(len(coverage_runs) > 0)

rel_names = [model_name] if model_name != 'ARM' \
    else [x for x in model_names if x not in ['occ_det', 'tennis', 'microcredit', 'potus']]

In [3]:
df = pd.DataFrame({'filename': coverage_runs, 'model_name': model_names})

# Drop the test models
df = df[~df['model_name'].str.contains('test')]

In [4]:

def compute_z_scores(loaded, reference_row_index=None, n_subset=None):
    
    if n_subset is not None:
        # Pick only the first n_subset
        loaded = loaded.iloc[:n_subset]
    
    if reference_row_index is not None:
        # Change reference
        reference_row = loaded.iloc[reference_row_index]
        loaded = loaded.copy()
                
        other_rows = loaded.loc[[x for x in loaded.index if x != reference_row.name]].copy()
        
        # This is pretty dumb -- better way?
        other_rows['reference_means'] = other_rows['reference_means'].apply(lambda _: reference_row['means'])
        other_rows['reference_freq_sds'] = other_rows['freq_sds'].apply(lambda _: reference_row['freq_sds'])
                
        loaded = other_rows
    
    z_scores = loaded.apply(
        lambda row: (row['means'] - row['reference_means']) / np.sqrt(
            row['reference_freq_sds']**2 + row['freq_sds']**2),
        axis=1)
    
    # Make them an array
    z_array = np.stack(z_scores.values)
    
    return z_array

def evaluate_z_scores(z_array, crit_prob=0.025):

    z_crit = norm.ppf(crit_prob)
    within_interval = np.abs(z_array) < np.abs(z_crit)
    within_ratio = within_interval.mean()
    
    return within_ratio

In [13]:
filename = df['filename'].iloc[0]
print(filename)
result = pd.read_pickle(filename)

./coverage_warm_starts_rerun/M_64/radon_no_pool_chr/coverage_results.pkl


In [34]:
# Looks like a single run is1000 different runs
print(len(result['seed']))
print(result.keys())

print('=======================\n')
#print(result.iloc[2]['freq_sds'])
print(result['M'])


100
Index(['means', 'seed', 'freq_sds', 'newton_step_norm', 'scipy_opt_result',
       'reference_means', 'reference_freq_sds', 'M',
       'reference_newton_step_norm', 'reference_scipy_opt_result'],
      dtype='object')

0     64
1     64
2     64
3     64
4     64
      ..
95    64
96    64
97    64
98    64
99    64
Name: M, Length: 100, dtype: int64


In [6]:
# Load the actual pickled results
df['loaded'] = df['filename'].apply(pd.read_pickle)

In [9]:
# Show some example data
df['loaded'].iloc[1].head()

,means,seed,freq_sds,newton_step_norm,scipy_opt_result,reference_means,reference_freq_sds,M,reference_newton_step_norm,reference_scipy_opt_result
0,"[87.01123, 2.905291, 6.013665, 0.55783266]",1000,"[0.1099612048616109, 0.004232627780388123, 0.2...",0.00000,"{'x': [87.01123, 2.905291, 6.013665, 0.5578326...","[86.95640093045265, 2.897581366961627, 5.97447...","[0.11779586115094012, 0.00431034592793851, 0.2...",64,0.0,"{'x': [86.9564, 2.8975813, 5.9744716, 0.559813..."
1,"[86.88435, 2.8961458, 5.932241, 0.5673275]",1001,"[0.11009873405193897, 0.004292656800942259, 0....",0.00000,"{'x': [86.88435, 2.8961458, 5.932241, 0.567327...","[86.95640093045265, 2.897581366961627, 5.97447...","[0.11779586115094012, 0.00431034592793851, 0.2...",64,0.0,"{'x': [86.9564, 2.8975813, 5.9744716, 0.559813..."
2,"[87.00213, 2.8992324, 5.7837462, 0.5744881]",1002,"[0.11242536325121924, 0.004290984416831309, 0....",0.00000,"{'x': [87.00213, 2.8992324, 5.7837462, 0.57448...","[86.95640093045265, 2.897581366961627, 5.97447...","[0.11779586115094012, 0.00431034592793851, 0.2...",64,0.0,"{'x': [86.9564, 2.8975813, 5.9744716, 0.559813..."
3,"[86.93819, 2.8927805, 6.268122, 0.57062894]",1003,"[0.11032773838013536, 0.004178320985602901, 0....",0.00001,"{'x': [86.93819, 2.8927805, 6.268122, 0.570628...","[86.95640093045265, 2.897581366961627, 5.97447...","[0.11779586115094012, 0.00431034592793851, 0.2...",64,0.0,"{'x': [86.9564, 2.8975813, 5.9744716, 0.559813..."
4,"[86.760765, 2.9056168, 5.761699, 0.559659]",1004,"[0.10810466393770593, 0.004249855760150295, 0....",0.00000,"{'x': [86.760765, 2.9056168, 5.761699, 0.55965...","[86.95640093045265, 2.897581366961627, 5.97447...","[0.11779586115094012, 0.00431034592793851, 0.2...",64,0.0,"{'x': [86.9564, 2.8975813, 5.9744716, 0.559813..."


A bit of explanation here: the field "loaded" now contains the rerun information. The column "means" lists the means for each rerun. The column "reference_means" lists the means for the reference run whose confidence interval we wish to evalute. And the column "reference_freq_sds" contains the estimated frequentist standard deviations.

In [8]:
# Look at the worst final Newton step for all reruns
df['worst_newton_step_norm'] = df['loaded'].apply(lambda x: x['newton_step_norm'].max())

In [ ]:
# Check convergence
np.log10(df['worst_newton_step_norm']).hist()

In [ ]:
# Drop models that likely didn't converge
print('Dropping:')
print(df[df['worst_newton_step_norm'] > 10**(-2)]['model_name'])

problematic =  df[df['worst_newton_step_norm'] >= 10**(-2)].copy()

df = df[df['worst_newton_step_norm'] < 10**(-2)].copy()

In [ ]:
# Also check scipy convergence message

def compute_frac_non_convergent(loaded):
    
    #return np.mean(loaded['scipy_opt_result'].apply(lambda x: x.status) != 0)
    return np.mean(loaded['scipy_opt_result'].apply(lambda x: x.message) != 'Optimization terminated successfully.')

def drop_non_converged(loaded):
    
    #return loaded[loaded['scipy_opt_result'].apply(lambda x: x.status) == 0]
    return loaded[loaded['scipy_opt_result'].apply(lambda x: x.message) == 'Optimization terminated successfully.']

# Check scipy convergence statuses
df['frac_not_converged'] = df['loaded'].apply(compute_frac_non_convergent)

# Drop these
df['loaded'] = df['loaded'].apply(drop_non_converged)

# Subset to the model names desired
df = df[df['model_name'].isin(rel_names)]

In [ ]:
df['frac_not_converged'].max()

In [ ]:
df['z_scores'] = df['loaded'].apply(partial(compute_z_scores, reference_row_index=REFERENCE_ROW_INDEX))

In [ ]:
df['frac_within'] = df['z_scores'].apply(evaluate_z_scores)

In [ ]:
df['M'] = df['loaded'].apply(lambda x: x['M'].iloc[0])

df['M'].value_counts()

In [ ]:
# 95% should lie within.
df[['model_name', 'M', 'frac_within']].sort_values('frac_within').head()

In [ ]:
# Turn the z-scores into "p-values"
df['p_vals'] = df['z_scores'].apply(norm.cdf)

In [ ]:
# Make them into one long vector
all_p_vals = np.concatenate(df['p_vals'].apply(lambda x: x.reshape(-1)).values)

In [ ]:
# Plot a histogram
f, ax = plt.subplots(1, 1)

_ = ax.hist(all_p_vals, density=True, bins=100)
ax.axhline(1, color='r')

f.set_size_inches(6, 4)
f.tight_layout()

#plt.savefig(f'overall_coverage_M={df["M"].iloc[0]}.png', dpi=300)

In [ ]:
# We can do a test to check for uniformity
stats.kstest(all_p_vals, stats.uniform(loc=0.0, scale=1.0).cdf)

In [ ]:
# We can do the same test by model
df.groupby('model_name').apply(
    lambda df: stats.kstest(df['p_vals'].iloc[0].reshape(-1), stats.uniform(loc=0.0, scale=1.0).cdf)[1]).sort_values().head(10)

In [ ]:
# Make sure this works as intended -- this one should not be rejected
stats.kstest(norm.cdf(np.random.randn(1000)), stats.uniform(loc=0., scale=1.).cdf)

In [ ]:
# Pick out a model of interest
model_to_check = 'occ_det'

model_p_vals = df[df['model_name'] == model_to_check]['p_vals'].iloc[0]
model_p_vals.shape

In [ ]:
df[df['model_name'] == model_to_check]

In [ ]:
df[df['model_name'] == model_to_check].iloc[0]['loaded'].head()

In [ ]:
plt.hist(model_p_vals.reshape(-1), density=True, bins=20)
plt.axhline(1.)

# plt.savefig(f'{model_to_check}_coverage_M={df["M"].iloc[0]}.png', dpi=300)

In [ ]:
model_z_scores = df[df['model_name'] == model_to_check]['z_scores'].iloc[0].reshape(-1)

In [ ]:
# Lastly, we can take a look at which z-score is most outlying
df['min_z_score'] = df['z_scores'].apply(lambda x: x.min())

In [ ]:
df.sort_values('min_z_score').head(10)

In [ ]:
worst_one = df.sort_values('min_z_score').iloc[0]

In [ ]:
worst_one